# Make SPU Dataset

In previous work, we focus on the problem of classifying a given image.

However, in a real problem setting, we are required to classify an SPU, rather than a single image.

An SPU has four or more images. It is a good practice to unify all prediction results of each image.

## Steps

### Overall 

+ data set prepration
    - training set
    - validation set
    - check overlap
+ make prediction
    - rewrite function `makePrediction`
    - voting algorithm, a class
+ model
    - use model from a checking point
    - train from scratch with new data set

### Details

***data set preparation.*** 

1. read `tmall_result.json`, `taobao_result.json` and `garbage_result.json`. filter out outliers.
1. randomly pick 70% SPU for training and 30% SPU for validation for each brand
2. 2% images in validation set as validation in learning curve.
2. ~~~store all these pic in relavant directory.~~~
    ```
    \dataset
        |----train
                |---- 4 sub-dir
        |----val
                |---- 4 sub-dir
        |----vallearn
    ```
3. check whether there is overlap between training set and validation set.
3. the images are saved in the same directory structure as before.

***training***

as usual, still train the classification of single image.    
report the prediction accuracy of individual image in learning curve.

***prediction***

vote algorithm:
+ sum weighted prob 
+ majority voting

1. make prediction for each image in validatio set
2. group images by SPU
3. apply vote algorithm
4. report SPU accuracy.

### Release

